In [1]:
# Import Libraries
import cv2
import numpy as np
import pandas as pd
import os
import struct

# Headers for different tables
meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

def readSplineData(fileName, nFrames):
    fCount = 0;
    spline = {}
    with open(fileName, "rb") as f:
        while (True) and (fCount < nFrames-1):
            fCount += 1
            gap, frameNumber = struct.unpack('>ii', f.read(struct.calcsize('>ii')))

            nPointsToRead =  struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" % (nPointsToRead)
            tempX = struct.unpack(fmt, f.read(struct.calcsize(fmt)))

            nPointsToRead =  struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" % (nPointsToRead)
            tempY = struct.unpack(fmt, f.read(struct.calcsize(fmt)))
            
            spline[frameNumber-1] = np.vstack((np.asarray(tempX).T, np.asarray(tempY).T))

    return spline

def readContourData(fileName, nFrames):    
    fCount = 0;
    contour = {}
    with open(fileName, "rb") as f:
        while (True) and (fCount < nFrames-1):
            fCount += 1
            frameNumber = struct.unpack('>i', f.read(struct.calcsize('>i')))

            nPointsToRead = struct.unpack('>i', f.read(struct.calcsize('>i')))            
            fmt = ">%dH" %(nPointsToRead)
            buff = f.read(struct.calcsize(fmt))
            tempX = struct.unpack(fmt, buff)
            
            nPointsToRead = struct.unpack('>i', f.read(struct.calcsize('>i')))
            fmt = ">%dH" %(nPointsToRead)
            buff = f.read(struct.calcsize(fmt))
            tempY = struct.unpack(fmt, buff)
            
            frameNumber = frameNumber[0]
            contour[frameNumber-1] = np.vstack((np.asarray(tempX).T, np.asarray(tempY).T))

    return contour


In [2]:
## Set path to the directory with tracker metadata and annotation
# test_dir = '../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_2/20180529_150807/Rawdata_20180417_082627_20180525_112343_A_133410/'
# test_string = "Rawdata_20180417_082627" ## Start frame in zero index

# test_dir = '../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_1/20180529_150721/Rawdata_20180417_084502_20180423_075225/'
# test_string = "Rawdata_20180417_084502" ## Start frame in zero index

test_dir = '../expts/trainingData_20180417_7c1/dataCollectedOn_20180417_grp_1/Rawdata_20180417_084502_20180423_075225/'
test_string = "Rawdata_20180417_084502" ## Start frame in zero index

## Set path to the directory with video file
video_path = '../expts/videos_20180417/'
video_file = os.path.join(video_path, test_string+".avi")
test_string_2 = str.split(test_string, '_')[2]

crop_size = 512
spot_size = 18.22
font = cv2.FONT_HERSHEY_SIMPLEX

## Read tracker metadata and annotation
for fs in os.listdir(test_dir):
    if 'Metadata' in fs:
        meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
    if 'Coordinates' in fs:
        coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)

numPoints = len(meta_data.index.values) + 1
for fs in os.listdir(test_dir):
    if 'Contour' in fs:
        contour = readContourData(os.path.join(test_dir, fs), numPoints)
    if 'SPLINE' in fs:        
        spline = readSplineData(os.path.join(test_dir, fs), numPoints)
        
if (meta_data.empty is False):

    ## OpenCV object for reading video files
    cap = cv2.VideoCapture(video_file)

    ## Total number of videos in the video file
    numberFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    ## Make the metadata frame umber start from 0 index
    meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] - 1
    meta_data.set_index('FrameNumber', inplace=True)

    ## Make the annotation frame umber start from 0 index
    coordinates = coordinates.round(0)
    start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
    coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'] - start_frame
    coordinates.set_index('FrameNumber', inplace=True)

    ## Inner join to take only the frames intersecting the annotation and tracker metadata
    ## Use when want to see only annotated frames 
#     meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='inner', suffixes=('_T', '_A'))

    ## Outer join to take only the frames intersecting the annotation and tracker metadata
    ## Use when want to see all frames 
    meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='outer', suffixes=('_T', '_A'))

    ## Replace NaN values with a high negative number
    meta_coord.fillna(-2000, inplace=True)

    ## Calculate distance between annotation and the tracker metadata
    distance = pd.DataFrame([], index=meta_coord.index.values)
    distance.index.name = 'FrameNumber'
    for head in distance_header:
        temp_x = (meta_coord[head+'_x_T'].values - meta_coord[head+'_x_A'].values)**2
        temp_y = (meta_coord[head+'_y_T'].values - meta_coord[head+'_y_A'].values)**2
        distance.loc[:, head+'_dist'] = np.sqrt(temp_x + temp_y)
    distance[distance > 2716] = np.nan

    ## Now append the distance table to the main table 'meta_coord'
    meta_coord = meta_coord.merge(distance, on='FrameNumber')
    
    ## Get ROC Statistics

    ## Initialization of VideoWriterObject to save the whole frame
    ## Uncomment below when want to save the whole frame
    outputFrameSize = np.multiply(np.ones((1920, 1920, 3), dtype=np.uint8), 255)
    height, width, layers = outputFrameSize.shape
    outputVideoFile = os.path.join(test_dir, '_'.join(str.split(test_string, '_')[0:3]) + '.avi')
    video = cv2.VideoWriter(outputVideoFile, 0, 5, (width, height))

    ## Initialization of VideoWriterObject to save the cropped frame
    outputFrameSize_crop = np.multiply(np.ones((crop_size, crop_size, 3), dtype=np.uint8), 255)
    height_crop, width_crop, layers_crop = outputFrameSize_crop.shape
    outputVideoFile_crop = os.path.join(test_dir, '_'.join(str.split(test_string, '_')[0:3]) + '_crop.avi')
    video_crop = cv2.VideoWriter(outputVideoFile_crop, 0, 5, (width_crop, height_crop))

    ## Check if the video is readable
    if cap.isOpened():
        for row_index, row in meta_coord.iterrows():
            this_frame = row_index + start_frame

            ## Set the frame number to be read from the Video
            cap.set(1, this_frame)

            ## Read the set frame from the Video
            ret, originalFrame = cap.read()
#             print this_frame,
#             print ret,

            ## Make copies of the frame for later use
            frame = originalFrame.copy()
            frame_overlay = originalFrame.copy()
            
            cv2.drawContours(frame, [contour[this_frame].T.astype(np.int32)], 0, (175,175,175), 3)
            cv2.polylines(frame, [spline[this_frame].T.astype(np.int32)], False, (125,125,225))

            ## Use head position as the crop center
            cropCenter_X = row['Head_x']
            cropCenter_Y = row['Head_y']

            ## Mark the head position with a '+' sign
            cv2.putText(frame_overlay, '+', (int(cropCenter_X), int(cropCenter_Y)), font, 0.65, (0, 0, 0), 3, cv2.LINE_AA)

            ####### Mark the detected positions by the tracker using a rectangle ######
            ## Tracker - MouthHook
            cv2.rectangle(frame_overlay, (int(row['MouthHook_x_T']) - int(spot_size/2),
                                  int(row['MouthHook_y_T']) - int(spot_size/2)),
                          (int(row['MouthHook_x_T']) + int(spot_size/2),
                           int(row['MouthHook_y_T']) + int(spot_size/2)),
                          color=(255, 0, 0), thickness=-1)

            ## Tracker - Left Dorsal Organ
            cv2.rectangle(frame_overlay, (int(row['LeftDorsalOrgan_x_T']) - int(spot_size/2),
                                  int(row['LeftDorsalOrgan_y_T']) - int(spot_size/2)),
                          (int(row['LeftDorsalOrgan_x_T']) + int(spot_size/2),
                           int(row['LeftDorsalOrgan_y_T']) + int(spot_size/2)),
                          color=(125, 125, 255), thickness=-1)

            ## Tracker - Right Dorsal Organ
            cv2.rectangle(frame_overlay, (int(row['RightDorsalOrgan_x_T']) - int(spot_size/2),
                                  int(row['RightDorsalOrgan_y_T']) - int(spot_size/2)),
                          (int(row['RightDorsalOrgan_x_T']) + int(spot_size/2),
                           int(row['RightDorsalOrgan_y_T']) + int(spot_size/2)),
                          color=(0, 255, 0), thickness=-1)

            ## Tracker - Left MouthHook
            cv2.rectangle(frame_overlay, (int(row['LeftMHhook_x_T']) - int(spot_size/4),
                                  int(row['LeftMHhook_y_T']) - int(spot_size/4)),
                          (int(row['LeftMHhook_x_T']) + int(spot_size/4),
                           int(row['LeftMHhook_y_T']) + int(spot_size/4)),
                          color=(100, 160, 220), thickness=-1)

            ## Tracker - Right MouthHook
            cv2.rectangle(frame_overlay, (int(row['RightMHhook_x_T']) - int(spot_size/4),
                                  int(row['RightMHhook_y_T']) - int(spot_size/4)),
                          (int(row['RightMHhook_x_T']) + int(spot_size/4),
                           int(row['RightMHhook_y_T']) + int(spot_size/4)),
                          color=(100, 200, 200), thickness=-1)

            ####### Mark the annotated positions using a circle #######
            ## Annotation - MouthHook
            cv2.circle(frame_overlay, (int(row['MouthHook_x_A']), int(row['MouthHook_y_A'])),
                       radius=5, color=(255, 0, 0), thickness=-1)

            ## Annotation - Left Dorsal Organ                
            cv2.circle(frame_overlay, (int(row['LeftDorsalOrgan_x_A']), int(row['LeftDorsalOrgan_y_A'])),
                       radius=5, color=(125, 125, 255), thickness=-1)

            ## Annotation - Right Dorsal Organ
            cv2.circle(frame_overlay, (int(row['RightDorsalOrgan_x_A']), int(row['RightDorsalOrgan_y_A'])),
                       radius=5, color=(0, 255, 0), thickness=-1)   

            ## Annotation - Left MouthHook                
            cv2.circle(frame_overlay, (int(row['LeftMHhook_x_A']), int(row['LeftMHhook_y_A'])),
                       radius=3, color=(100, 160, 220), thickness=-1)

            ## Annotation - Right MouthHook
            cv2.circle(frame_overlay, (int(row['RightMHhook_x_A']), int(row['RightMHhook_y_A'])),
                       radius=3, color=(100, 200, 200), thickness=-1)   

            ## Crop the frame
            crop_x = int(max(0, cropCenter_X-int(crop_size/2)))
            crop_y = int(max(0, cropCenter_Y-int(crop_size/2)))

            frame_crop = frame[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size]
            frame_crop_overlay = frame_overlay[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size]

            ## Display statistics at the top-right corner
            cv2.putText(frame_crop, '%03d'%(this_frame), (360, 40), font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, 'Conf', (415, 40), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, 'Err', (465, 40), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.putText(frame_crop, '%s: %02d %2.1f' % (' MH', row['MouthHook_votes'], row['MouthHook_dist']), (360, 60), font, 0.65, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s: %02d %2.1f' % ('LDO', row['LeftDorsalOrgan_votes'], row['LeftDorsalOrgan_dist']), (360, 80), font, 0.65, (125, 125, 255), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s: %02d %2.1f' % ('RDO', row['RightDorsalOrgan_votes'], row['RightDorsalOrgan_dist']), (360, 100), font, 0.65, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s: %02d %2.1f' % ('LMH', row['LeftMHhook_votes'], row['LeftMHhook_dist']), (360, 120), font, 0.65, (100, 160, 220), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s: %02d %2.1f' % ('RMH', row['RightMHhook_votes'], row['RightMHhook_dist']), (360, 140), font, 0.65, (100, 200, 200), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, ' KP: %02d' % (row['Num_Key_points']), (362, 160), font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, 'Spot: %02d um' % (spot_size*2.75), (355, 180), font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)

            ## To make the rectangle and circle slightly transparent
            alpha = 0.3
            cv2.addWeighted(frame_crop_overlay, alpha, frame_crop, 1 - alpha, 0, frame_crop)
            cv2.addWeighted(frame_overlay, alpha, frame, 1 - alpha, 0, frame)
            video_crop.write(frame_crop)

            ## Uncomment when saving the complete frame
#             video.write(frame)          

    cv2.destroyAllWindows()
    video_crop.release()

    ## Uncomment when saving the complete frame    
#     video.release()